In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

import numpy as np
from sklearn.utils import shuffle
import nibabel as nib 
import glob
from matplotlib import pyplot as plt
%matplotlib inline
!pip install nilearn
import nilearn as nil
import cv2

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 2.5MB 3.4MB/s 


In [ ]:
#Read training set
corr_imgs = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Corrected Dataset/*')) 
labels = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/*'))


['drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/BiasField_acpc_dc (1).nii.gz',
 'drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/BiasField_acpc_dc (10).nii.gz',
 'drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/BiasField_acpc_dc (11).nii.gz',
 'drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/BiasField_acpc_dc (12).nii.gz',
 'drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/BiasField_acpc_dc (13).nii.gz']

In [ ]:
N = 64
total_imgs = 0

# Take each labels multiply with 40 skull stripped bias corrected images

for l in range(len(labels)): # Iterating through each label
  label = []
  a = nib.load(labels[l])
  a = a.get_fdata()
  for k in range(a.shape[0]):                                                                                                  
    temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  for k in range(a.shape[1]):                                                                                                  
    temp = cv2.resize(a[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  for k in range(a.shape[2]):                                                                                                  
    temp = cv2.resize(a[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)
  
  label = np.asarray(label)  
  label = label.reshape(-1,N,N,1)  # (260 * 64 * 64 * 1)

  for c in range(len(corr_imgs)): # Iterating through corrected images
    image = []
    a = nib.load(corr_imgs[c])
    a = a.get_fdata()

    for k in range(a.shape[0]):                                                                                                  
      temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                          
      image.append(temp)

    for k in range(a.shape[1]):                                                                                                  
      temp = cv2.resize(a[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR)                                          
      image.append(temp)

    for k in range(a.shape[2]):                                                                                                  
      temp = cv2.resize(a[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR)                                          
      image.append(temp)
    
    image = np.asarray(image) 
    image = image.reshape(-1,N,N,1)  # (260 * 64 * 64 * 1)

    #for i in range(len(label)):  
    #  for j in range(N):
    #    for k in range(N):
    #      if (image[i,j,k,0] == 0):  # Based on Skull Stripped image, making the bias zero
    #        label[i,j,k,0] = 0  

    temp_img = np.multiply(label,image) # Element wise multiplication of bias and corrected MRI  

    final_img = np.empty([260,311,260]) # converting the image from (260 * 64 * 64 * 1) to (260*311*260)
    for idx in range(0,260):
      img = temp_img[idx, :, :,0]
      img_resized = cv2.resize(img, (260, 311), interpolation=cv2.INTER_CUBIC)
      final_img[idx,:,:] = img_resized

    total_imgs = total_imgs + 1

    # Saving Images in respective folder
    final_img = nil.image.new_img_like(nib.load(corr_imgs[c]),final_img,copy_header=True)
    nib.save(final_img,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Test_Set/TestImages/' + 'input_train-' + str(total_imgs) + '.nii.gz')
    
    just_for_ref_corr = nib.load(corr_imgs[c]).get_fdata()
    just_for_ref_corr = nil.image.new_img_like(nib.load(corr_imgs[c]),just_for_ref_corr, copy_header=True)
    nib.save(just_for_ref_corr,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Test_Set/TestCorrected_GroundTruth/'  + 'corrected_train-'  + str(total_imgs) +'.nii.gz')
  
    label_save = nib.load(labels[l]).get_fdata()
    label_save = nil.image.new_img_like(nib.load(labels[l]),label_save,copy_header=True)
    nib.save(label_save,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Test_Set/TestLabels/'  + 'label_train-'  + str(total_imgs) +'.nii.gz')
  
    


In [ ]:
N= 64

# Uncompress the Augmented Images to .npz format
imgs = sorted(glob.glob('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Training_Set/Images_nii_gz/*'))
labels = sorted(glob.glob('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Training_Set/Labels_nii_gz/*'))

#For Labels
for l in range(len(labels)): # Iterating through each label
  label = []
  a = nib.load(labels[l])
  a = a.get_fdata()
  for k in range(a.shape[0]):                                                                                                  
    temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  for k in range(a.shape[1]):                                                                                                  
    temp = cv2.resize(a[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  for k in range(a.shape[2]):                                                                                                  
    temp = cv2.resize(a[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  label = np.asarray(label)
  label = label.reshape(-1,N,N,1) 

  np.savez_compressed('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Training_Set/Labels_npz/label-'+ str(l+1) + '.npz', label)


# For Images
for l in range(len(imgs)): # Iterating through each label
  image = []
  a = nib.load(imgs[l])
  a = a.get_fdata()
  for k in range(a.shape[0]):                                                                                                  
    temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    image.append(temp)

  for k in range(a.shape[1]):                                                                                                  
    temp = cv2.resize(a[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    image.append(temp)

  for k in range(a.shape[2]):                                                                                                  
    temp = cv2.resize(a[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    image.append(temp)

  image = np.asarray(image)
  image = image.reshape(-1,N,N,1) 
  np.savez_compressed('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/Training_Set/Images_npz/input-'+ str(l+1) + '.npz', image)

In [ ]:
imgs = sorted(glob.glob('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TrainAgain/InputImages_npz/*'))
labels = sorted(glob.glob('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TrainAgain/Labels_npz/*'))


#Finding minimum and Maximum values in images
Max = float("-inf")
Min = float("inf")
for i in range(len(imgs)):
  images = np.load(imgs[i])['arr_0']

  if (np.max(images) > Max):
    Max = np.max(images)

  if (np.min(images) < Min):
    Min = np.min(images)


#Finding minimum and Maximum values in labels
Max_lab = float("-inf")
Min_lab = float("inf")

for i in range(len(labels)):
  lab = np.load(labels[i])['arr_0']

  if (np.max(lab) > Max_lab):
    Max_lab = np.max(lab)

  if (np.min(lab) < Min_lab):
    Min_lab = np.min(lab)  


In [ ]:
#Normalize Images
Max_norm  = 0
Min_norm  = 0

for i in range(len(imgs)):
  images = np.load(imgs[i])['arr_0']
  images = (images - Min) / (Max - Min)
  if (np.max(images) > Max_norm):
    Max_norm = np.max(images)

  if (np.min(images) < Min_norm):
    Min_norm = np.min(images) 

  #print(Max_norm, "  ", Min_norm)
  np.savez_compressed('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TrainAgain/InputImages_npz_normalize/' + imgs[i][124:] , images)

In [ ]:
#Normalize Labels
Max_norm  = 0
Min_norm  = 0

for i in range(len(labels)):

  lab = np.load(labels[i])['arr_0']

  lab = (lab - Min_lab) / (Max_lab - Min_lab)
  
  if (np.max(images) > Max_norm):
    Max_norm = np.max(lab)

  if (np.min(images) < Min_norm):
    Min_norm = np.min(lab) 

 # print(Max_norm, "  ", Min_norm)

  np.savez_compressed('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TrainAgain/Labels_npz_normalize/' + labels[i][119:] , lab)

In [ ]:
#Let's stop this and do it in my way
corr_imgs = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Corrected Dataset/*'))   #reading corr mris
labels = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/*')) # reading labels
labels = labels[5:40]
total_imgs = 0
N = 64
for l in range(len(labels)): # Iterating through each label
  
  biasField = nib.load(labels[l])
  biasField = biasField.get_fdata()

  label = []
  for k in range(biasField.shape[0]):                                                                                                  
    temp = cv2.resize(biasField[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  for k in range(biasField.shape[1]):                                                                                                  
    temp = cv2.resize(biasField[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  for k in range(biasField.shape[2]):                                                                                                  
    temp = cv2.resize(biasField[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  label = np.asarray(label)
  label = label.reshape(-1,N,N,1)

  for c in range(len(corr_imgs)): # Iterating through corrected images
    image = []
    corrImage = nib.load(corr_imgs[c])
    corrImage = corrImage.get_fdata()

    InputImage = np.multiply(biasField, corrImage)
    total_imgs = total_imgs +1

    image = []
    for k in range(InputImage.shape[0]):                                                                                                  
      temp = cv2.resize(InputImage[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
      image.append(temp)

    for k in range(InputImage.shape[1]):                                                                                                  
      temp = cv2.resize(InputImage[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
      image.append(temp)

    for k in range(InputImage.shape[2]):                                                                                                  
      temp = cv2.resize(InputImage[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR)                                            
      image.append(temp)

    image = np.asarray(image)
    image = image.reshape(-1,N,N,1)
  
    np.savez_compressed('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TrainAgain/Labels_npz/'+'label-train'+ str(total_imgs) + '.npz', label)
    np.savez_compressed('drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TrainAgain/InputImages_npz/'+'input-train'+ str(total_imgs) + '.npz', image)








In [ ]:
#Test Set -- Let's stop this and do it in my way 
corr_imgs = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Corrected Dataset/*'))   #reading corr mris
labels = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/*')) # reading labels
labels = labels[:5]
total_imgs = 0
N = 64
for l in range(len(labels)): # Iterating through each label
  
  biasField = nib.load(labels[l])
  biasField = biasField.get_fdata()

  for c in range(len(corr_imgs)): # Iterating through corrected images
    corrImage = nib.load(corr_imgs[c])
    corrImage = corrImage.get_fdata()

    InputImage = np.multiply(biasField, corrImage)
    total_imgs = total_imgs +1

    # Saving Images in respective folder
    InputImage = nil.image.new_img_like(nib.load(corr_imgs[c]),InputImage, copy_header=True)
    nib.save(InputImage,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TestAgain/Images/' + 'input_train-' + str(total_imgs) + '.nii.gz')
    
    just_for_ref_corr = nib.load(corr_imgs[c]).get_fdata()
    just_for_ref_corr = nil.image.new_img_like(nib.load(corr_imgs[c]),just_for_ref_corr, copy_header=True)
    nib.save(just_for_ref_corr,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TestAgain/CorrectedImages/'  + 'corrected_train-'  + str(total_imgs) +'.nii.gz')
  
    label_save = nib.load(labels[l]).get_fdata()
    label_save = nil.image.new_img_like(nib.load(labels[l]),label_save,copy_header=True)
    nib.save(label_save,'drive/My Drive/DR_NonUniformityCorrection/Augmented_Training/Augment_MRI_Images/Lets_start_again/TestAgain/Labels/'  + 'label_train-'  + str(total_imgs) +'.nii.gz')
  


